In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
import math
import random

def dot(x, y):
    if isinstance(x[0],(int,float)):
        if not isinstance(y[0],(int,float)) or len(x)!=len(y):
            raise ValueError("dimensions don't match")
        ans = 0
        for i in range(len(x)):
            ans+=x[i]*y[i]
        return ans
    if isinstance(y[0],(int,float)):
        if not isinstance(x[0],(int,float)) or len(x)!=len(y):
            raise ValueError("dimensions don't match")
    if len(x[0]) != len(y):
        raise ValueError("Number of columns in matrix x must equal number of rows in matrix y.")

    rows_x, cols_x = len(x), len(x[0])
    rows_y, cols_y = len(y), len(y[0])

    result = [[0 for _ in range(cols_y)] for _ in range(rows_x)]
    for i in range(rows_x):
        for j in range(cols_y):
            for k in range(cols_x):
                result[i][j] += x[i][k] * y[k][j]
    return result

def sigmoid(x):
    # Activation function for sigmoid is f(x) = 1/(1+e^(-x))
    if isinstance(x,(int,float)):
        return 1/(1+math.exp(-x))

    if isinstance(x,list):
        return [sigmoid(elem) for elem in x]

def relu(x):
    """
    ReLU activation function.
    f(x) = max(0, x)
    """
    if isinstance(x, (int, float)):
        return max(0, x)

    if isinstance(x, list):
        return [relu(elem) for elem in x]

    if isinstance(x, list) and isinstance(x[0], list):
        return [relu(row) for row in x]

def leaky_relu(x, alpha=0.01):
    """
    Leaky ReLU activation function.
    f(x) = max(alpha * x, x)
    """
    if isinstance(x, (int, float)):
        return max(alpha * x, x)

    if isinstance(x, list):
        return [leaky_relu(elem, alpha) for elem in x]

    if isinstance(x, list) and isinstance(x[0], list):
        return [leaky_relu(row, alpha) for row in x]

def elu(x, alpha=1.0):
    """
    ELU activation function.
    f(x) = x for x > 0, alpha * (exp(x) - 1) for x <= 0
    """
    if isinstance(x, (int, float)):
        return x if x > 0 else alpha * (math.exp(x) - 1)

    if isinstance(x, list):
        return [elu(elem, alpha) for elem in x]

    if isinstance(x, list) and isinstance(x[0], list):
        return [elu(row, alpha) for row in x]

def tanh(x):
    """
    Tanh activation function.
    f(x) = (exp(x) - exp(-x)) / (exp(x) + exp(-x))
    """
    if isinstance(x, (int, float)):
        return math.tanh(x)

    if isinstance(x, list):
        return [tanh(elem) for elem in x]

    if isinstance(x, list) and isinstance(x[0], list):
        return [tanh(row) for row in x]

def softmax(x):
    """
    Softmax activation function (only works on lists or 2D matrices).
    f(x)_i = exp(x_i) / sum(exp(x_j) for all j)
    """
    if isinstance(x, list):
        exp_x = [math.exp(i) for i in x]
        total = sum(exp_x)
        return [i / total for i in exp_x]

    if isinstance(x, list) and isinstance(x[0], list):
        return [softmax(row) for row in x]

def swish(x):
    """
    Swish activation function.
    f(x) = x * sigmoid(x)
    """
    if isinstance(x, (int, float)):
        return x * (1 / (1 + math.exp(-x)))

    if isinstance(x, list):
        return [swish(elem) for elem in x]

    if isinstance(x, list) and isinstance(x[0], list):
        return [swish(row) for row in x]

def gelu(x):
    """
    GELU activation function.
    f(x) = 0.5 * x * (1 + tanh(sqrt(2 / pi) * (x + 0.044715 * x^3)))
    """
    if isinstance(x, (int, float)):
        return 0.5 * x * (1 + math.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * x**3)))

    if isinstance(x, list):
        return [gelu(elem) for elem in x]

    if isinstance(x, list) and isinstance(x[0], list):
        return [gelu(row) for row in x]

def softplus(x):
    """
    Softplus activation function.
    f(x) = ln(1 + exp(x))
    """
    if isinstance(x, (int, float)):
        return math.log(1 + math.exp(x))

    if isinstance(x, list):
        return [softplus(elem) for elem in x]

    if isinstance(x, list) and isinstance(x[0], list):
        return [softplus(row) for row in x]

def hard_sigmoid(x):
    """
    Hard Sigmoid activation function (simplified version of sigmoid).
    f(x) = clip((x + 1) / 2, 0, 1)
    """
    if isinstance(x, (int, float)):
        return min(max((x + 1) / 2, 0), 1)

    if isinstance(x, list):
        return [hard_sigmoid(elem) for elem in x]

    if isinstance(x, list) and isinstance(x[0], list):
        return [hard_sigmoid(row) for row in x]

def hard_swish(x):
    """
    Hard Swish activation function.
    f(x) = x * clip(x + 3, 0, 6) / 6
    """
    if isinstance(x, (int, float)):
        return x * min(max(x + 3, 0), 6) / 6

    if isinstance(x, list):
        return [hard_swish(elem) for elem in x]

    if isinstance(x, list) and isinstance(x[0], list):
        return [hard_swish(row) for row in x]

def sigmoid_derivative(x):
    if isinstance(x,(int,float)):
        s = sigmoid(x)
        return s * (1-s)
    if isinstance(x,list):
        return [sigmoid_derivative(elem) for elem in x]

    raise ValueError("Unsupported type for sigmoid_derivative")




In [ ]:
import math

def mean_squared_error(y_true, y_pred):
    return sum((yt - yp) ** 2 for yt, yp in zip(y_true, y_pred)) / len(y_true)

def cross_entropy_loss(y_true, y_pred):
    epsilon = 1e-15  # to avoid log(0)
    y_pred = [max(min(yp, 1 - epsilon), epsilon) for yp in y_pred]
    return -sum(yt * math.log(yp) + (1 - yt) * math.log(1 - yp) for yt, yp in zip(y_true, y_pred)) / len(y_true)

# Example usage:
y_true = [1, 0, 1, 1]
y_pred = [0.9, 0.1, 0.8, 0.7]

#print("MSE:", mean_squared_error(y_true, y_pred))
#print("Cross-Entropy Loss:", cross_entropy_loss(y_true, y_pred))


def mean_squared_error_derivative(y_true, y_pred):
    return [(2 / len(y_true)) * (yp - yt) for yt, yp in zip(y_true, y_pred)]

def cross_entropy_loss(y_true, y_pred):
    epsilon = 1e-15  # to avoid log(0)
    y_pred = [max(min(yp, 1 - epsilon), epsilon) for yp in y_pred]
    return -sum(yt * math.log(yp) + (1 - yt) * math.log(1 - yp) for yt, yp in zip(y_true, y_pred)) / len(y_true)

def cross_entropy_loss_derivative(y_true, y_pred):
    epsilon = 1e-15  # to avoid division by zero
    y_pred = [max(min(yp, 1 - epsilon), epsilon) for yp in y_pred]
    return [-(yt / yp) + (1 - yt) / (1 - yp) for yt, yp in zip(y_true, y_pred)]

In [ ]:

class Neuron:
    def __init__(self, weights, bias, activation = sigmoid, activation_grad = sigmoid_derivative):
        self.weights = weights
        self.bias = bias
        self.activation = activation
        self.activation_grad = activation_grad
    def feed_forward(self, inputs):
        self.input_cache = inputs
        total = dot(self.weights, inputs)
        # If total is scalar, you directly add bias
        if isinstance(total, (int, float)):
            self.output = self.activation(total + self.bias)
            return self.output

        # If total is a list (or higher dimension), we must handle broadcasting and apply bias element-wise
        if isinstance(total, list):
            self.output = self.activation(self.apply_bias_elementwise(total,self.bias))
            return self.output

    def apply_bias_elementwise(self, total, bias):
        # For element-wise addition of bias based on the dimensionality of total and bias
        if isinstance(total, list):
            if isinstance(total[0], (list, tuple)):  # Handle case for 2D or higher
                return [self.apply_bias_elementwise(row, bias) for row in total]
            else:  # Handle 1D case (list of scalars)
                if isinstance(bias, (list, tuple)):  # Bias should be of the same length as total for 1D case
                    return [total[i] + bias[i] for i in range(len(total))]
                else:  # Scalar bias
                    return [elem + bias for elem in total]

        # This would handle scalar `total`, but we expect to handle list or higher only.
        raise ValueError("The total should be a list, tuple, or higher dimensional structure.")
            # assume it is a list here

    def multiply_element_wise(self, a,b):
        if isinstance(a,(int,float)) and isinstance(b,(int,float)):
            return a*b
        return [self.multiply_element_wise(a[i],b[i]) for i in range(len(a))]
    def backward(self, grad):
        # Compute gradient with respect to weights and bias
        grad_activation = self.activation_grad(self.output)

        grad_total = self.multiply_element_wise(grad, grad_activation)

        self.grad_weights = [grad_total * inp for inp in self.input_cache]
        self.grad_bias = grad_total

        grad_inputs = [grad_total * w for w in self.weights]

        return grad_inputs

    def update_params(self, learning_rate = 0.01):
        self.weights = [w - learning_rate * grad_w for w, grad_w in zip(self.weights, self.grad_weights)]
        self.bias -= learning_rate * self.grad_bias


class Dense:
    def __init__(self, num_neurons, input_size, activation = sigmoid, activation_grad = sigmoid_derivative):
        self.num_neurons = num_neurons
        self.input_size = input_size
        self.activation = activation
        self.activation_grad = activation_grad
        self.initialize_neurons()

    def initialize_neurons(self):
        self.neurons = []
        scale = 1
        for _ in range(self.num_neurons):
            weights = [random.uniform(-scale,scale) for _ in range(self.input_size)]
            bias = random.uniform(-scale,scale)
            neuron = Neuron(weights, bias, self.activation, self.activation_grad)
            self.neurons.append(neuron)

    def forward(self, inputs):
        output = [neuron.feed_forward(inputs) for neuron in self.neurons]
        return output

    def backward(self, grads):
        grad_inputs = [0 for _ in range(self.input_size)]
        for i, neuron in enumerate(self.neurons):
            grad_input = neuron.backward(grads[i])
            grad_inputs = [grad_inputs[j] + grad_input[j] for j in range(self.input_size)]
        return grad_inputs

    def update_params(self, learning_rate = 0.01):
        for neuron in self.neurons:
            neuron.update_params(learning_rate)


class Conv2D:
    def __init__(self, num_filters, filter_size, input_shape, stride = 1, padding = 'valid', activation_function = None):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.input_shape = input_shape
        self.stride = stride
        self.padding = padding
        self.activation_function = None

        self.initialize_filters()
        self.biases = [random.random() for _ in range(self.num_filters)]

    def initialize_filters(self):
        self.filters = []
        for _ in range(self.num_filters):
            filter_mat = []
            for _ in range(self.filter_size[0]):
                filter_mat.append([[random.random() for _ in range(self.input_shape[2])] for _ in range(self.filter_size[1])])
            self.filters.append(filter_mat)

    def apply_padding(self,inputs):
        if self.padding == 'same':
            pad_height = (self.filter_size[0]-1)//2
            pad_width = (self.filter_size[1]-1)//2
            padded_input = []

            for channel in range(len(inputs[0][0])):
                padded_channel =[[0] * (len(inputs[0]) + 2*pad_width)
                                 for _ in range(len(inputs)+2*pad_height)]
                for i in range(len(inputs)):
                    for j in range(len(inputs[0])):
                        padded_channel[i+pad_height][j+pad_width]=inputs[i][j][channel]
                    padded_input.append(padded_channel)
            return padded_input
        return inputs

    def element_wise_product_sum(self, slice_input, filter, bias):
        total = 0
        for i in range(len(filter)):
            for j in range(len(filter[0])):
                for k in range(len(filter[0][0])):
                    total += slice_input[i][j][k]*filter[i][j][k]
        return total+bias

    def convolve(self,inputs):
        inputs = self.apply_padding(inputs)

        input_height = len(inputs)
        input_width = len(inputs[0])
        input_depth = len(inputs[0][0])

        output_height = (input_height-self.filter_size[0])//self.stride + 1
        output_width = (input_width - self.filter_size[1])//self.stride + 1

        output = [[[0.0 for _ in range(self.num_filters)] for _ in range(output_width)] for _ in range(output_height)]

        for i in range(output_height):
            for j in range(output_width):
                for f in range(self.num_filters):
                    vertical_start = i*self.stride
                    vertical_end = vertical_start + self.filter_size[0]
                    horizontal_start = j*self.stride
                    horizontal_end = horizontal_start + self.filter_size[1]

                    slice_input = [
                        inputs[vertical_start+x][horizontal_start:horizontal_end]
                        for x in range(self.filter_size[0])
                    ]

                    value = self.element_wise_product_sum(slice_input, self.filters[f], self.biases[f])

                    if self.activation_function:
                        value = self.activation_function(value)
                output[i][j][f]=value
        return output


In [ ]:



class NeuralNetwork:
    def __init__(self):
        self.layers = []

    def add_layer(self, layer):
        self.layers.append(layer)

    def forward(self, inputs):
        output = inputs
        for layer in self.layers:
            output = layer.convolve(output) if isinstance(layer, Conv2D) else layer.forward(output)
        return output

    def backward(self, loss_grad):
        grad = loss_grad
        for layer in reversed(self.layers):
            grad = layer.backward(grad)
            #print(f'grad at layer is {grad}')

    def update_params(self, learning_rate = 0.01):
        for layer in self.layers:
            layer.update_params(learning_rate)

    def train(self, inputs, targets, loss_function, loss_grad_function, epochs, learning_rate = 0.01,):
        for epoch in range(epochs):
            print(f'epoch {epoch+1}/{epochs}')
            loss = 0
            #print(f'enumerating over data')
            for i, input in enumerate(inputs):
                target = targets[i]
                #if i % 100 == 0:
                    #print(f'before prediction on iteration {i}')
                prediction = self.forward(input)
                #if i % 100 == 0:
                    #print(f'after prediction on iteration {i}')
                loss += loss_function(target, prediction)
                loss_grad = loss_grad_function(target, prediction)
                #print(f'loss grad is {loss_grad}')
                self.backward(loss_grad)
            print(f'epoch {epoch+1}/{epochs}, loss: {loss/len(inputs)}')

            self.update_params(learning_rate)

In [ ]:
# Load dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# Reduce the size of the dataset
# Choose a random subset of 3000 training samples and 500 test samples
train_indices = np.random.choice(X_train.shape[0], 3000, replace=False)
test_indices = np.random.choice(X_test.shape[0], 500, replace=False)

X_train = X_train[train_indices].reshape(3000, -1) / 255.0  # Flatten and normalize
X_test = X_test[test_indices].reshape(500, -1) / 255.0     # Flatten and normalize
y_train = y_train[train_indices]
y_test = y_test[test_indices]

# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False)
y_train_encoded = encoder.fit_transform(y_train.reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.reshape(-1, 1))

# Split 3000 training samples into train and validation sets
X_train, X_val, y_train_encoded, y_val_encoded = train_test_split(
    X_train, y_train_encoded, test_size=0.1, random_state=42
)

# Convert datasets into lists if required
X_train = list(X_train)
X_val = list(X_val)
X_test = list(X_test)
y_train_encoded = list(y_train_encoded)
y_val_encoded = list(y_val_encoded)
y_test_encoded = list(y_test_encoded)

In [ ]:
print(f'{len(X_train)}')
print(f'{type(X_train)}')

2700
<class 'list'>


In [ ]:
nn = NeuralNetwork()
layer_one = Dense(num_neurons=128, input_size=784)
layer_two = Dense(num_neurons=64, input_size=128)
layer_three = Dense(num_neurons=10, input_size=64)

nn.add_layer(layer_one)  # First hidden layer with sigmoid
nn.add_layer(layer_two)   # Second hidden layer with sigmoid
nn.add_layer(layer_three)

In [ ]:
nn.train(
    inputs = X_train,
    targets = y_train_encoded,
    loss_function = cross_entropy_loss,
    loss_grad_function= cross_entropy_loss_derivative,
    epochs = 100,
    learning_rate = 0.01
)

epoch 1/100
epoch 1/100, loss: 1.3230049212342592
epoch 2/100
epoch 2/100, loss: 0.6973699541972684
epoch 3/100
epoch 3/100, loss: 0.6694519772908976
epoch 4/100
epoch 4/100, loss: 0.655308278964443
epoch 5/100
epoch 5/100, loss: 0.6470332325151484
epoch 6/100
epoch 6/100, loss: 0.6422884217839918
epoch 7/100
epoch 7/100, loss: 0.6400223855233625
epoch 8/100
epoch 8/100, loss: 0.6396588876567321
epoch 9/100
epoch 9/100, loss: 0.6408403656045183
epoch 10/100
epoch 10/100, loss: 0.6433244887803393
epoch 11/100
epoch 11/100, loss: 0.6469356037390133
epoch 12/100
epoch 12/100, loss: 0.6515394255286452
epoch 13/100
epoch 13/100, loss: 0.6570288048012282
epoch 14/100
epoch 14/100, loss: 0.6633152540559477
epoch 15/100
epoch 15/100, loss: 0.670323676271772
epoch 16/100
epoch 16/100, loss: 0.6779889705610024
epoch 17/100
epoch 17/100, loss: 0.6862537842547518
epoch 18/100
epoch 18/100, loss: 0.6950669873116261
epoch 19/100
epoch 19/100, loss: 0.7043826122948462
epoch 20/100
epoch 20/100, loss: